# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    print("網頁更新中...")
    
    # Wait to load the page.
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        print("到達頁面底端")
        break

    last_height = new_height

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [4]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [5]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 台股早盤上漲逾百點！台積電、鴻海帶頭衝　台股站上11,700點

▲ 台股。（圖／記者湯興漢攝）
記者李瑞瑾／台北報導
華爾街從武漢肺炎疫情的陰霾中走出，美股四大指數續揚，道瓊收盤大漲近500點、站回29,000點大關，標普和那指再創新高。台股今（6）日亦受美股激勵，開盤上漲31點、以11,605點開出。
電子三雄中，台積電（2330）以329.5元開出、上漲0.61%；鴻海（2317）開在82.5元、上漲0.36%；股王大立光（3008）以4800元開出、下跌0.62%。
截至上午9點18分，台積電、鴻海擴大漲勢來到1%以上，推升加權指數來到11,703點、上漲130點。
新型冠狀病毒疫苗的研發釋放正面消息，市場押寶中國武漢肺炎疫情不會重創全球經濟成長，華爾街投資人漸從武漢肺炎的惶恐中走出，加上美國就業數據傳出利多，美股連續兩日走高，道瓊昨日收盤大漲近500點、漲幅超過1%，標普和那指再創新高。
美股道瓊指數上漲483.22點，或1.68%，收29,290.85點；那斯達克指數上漲40.71點，或0.43%，收9,508.68點；標普 500 指數上漲37.4點，或1.13%，收3,334.99點；費城半導體指數上漲40.87點，或2.19%，收1, 909.52點。
----------------------------------------------------------------------

[1] 新台幣早盤勁揚逾1角！　一度再現29字頭

▲ 新台幣兌美元今天開盤升值0.2分。（圖／路透社）
記者李瑞瑾／台北報導
台北外匯市場新台幣兌美元匯率今（6）日開盤升值0.2分、以30.15元兌1美元開出，開盤後不久即升值超過1角，最高一度來到29.99元兌1美元。
新台幣兌美元昨（5）日收在30.152兌1美元；今天開盤則升值0.2分，以30.15元兌1美元開出。
新台幣開盤後不久即大升逾1角，截至上午9點07分，新台幣來到30.009元兌1美元，升值1.43角。
---------------------------------------------------------------------

記者姚惠茹／台北報導
連接線廠映興電子（3597）董事長賴柄源今（5）日表示，雖然電子產業受到中美貿易戰和武漢肺炎疫情的衝擊，但是映興受惠3年前就開始佈局的泰國廠啟動量產，以及轉單效應，樂觀看待今年營運，尤其是車用和綠能2大產品線可望成為今年的營運成長動能，並預計在3月初掛牌上櫃。
賴柄源表示，映興電子主要產品為連接器與連接線組，產品應用在工業應用、車裝電子、綠能節能、健康照護等四大產業，目前客戶多為國內知名上市櫃公司，其中透過專利佈局保護的防水連接器件模組與移動式太陽能智慧儲能系統，深受國內外市場肯定，正逐步加大營運動能。
▲左起為映興電子財務經理關慧貞、發言人林宜嫻、董事長賴柄源、研發副總賴啟益、技術部經理鄭凱陽。（圖／記者姚惠茹攝）
賴柄源指出，映興電子原本在台中和昆山設廠，產能佔比約為6:4，但從3年前開始建置泰國廠，前年正式投產，不僅增加當地接單數量，也正好在中美貿易戰和武漢肺炎疫情擴散之際，透過彈性調度，取代昆山廠產能降低衝擊，並期許今年下半年的產能佔比達6:3:1，藉此進軍東南亞市場。
賴柄源認為，今年車用和綠能會有比較大的成長，其中車用方面，防水連接器應用在電動代步車及電動自行車有很大的成長空間，目前公司團隊已經成立新的事業單位積極開發國際市場，像是美國、歐洲、日本都有新的客戶洽談，預計今年第2季、第3季就可以看到成果。
▲映興電子董事長賴柄源。（圖／記者姚惠茹攝）
綠能方面，賴柄源表示，移動式太陽能智慧儲能系統目前在加州的需求很大，因此映興電子就在美國設立子公司，增加服務據點，現在約有850台的銷售量，而主力客戶在北美則有1000台的銷售量，未來計畫與美國當地品牌合作，作為前進國際市場的基地。
映興電子2019年前3季營收5.98億元，稅後淨利為2438萬元，每股盈餘為0.8元；累計2019年合併營收為7.78億元，年減1.71%，賴柄源表示，主要是計算期間不同，以及中美貿易戰衝擊影響，導致移動式太陽能智慧儲能系統出貨放緩，但今年車用、綠能可望推升營運成長。
----------------------------------------------------------------------

[17] 謝金河：全世界都在看Tesla表演！空頭慘賠數十億美元　分析師喊股價漲到7千美元

▲Tesla執行長伊隆·馬斯克(Elon Musk)。（

▲保險局自2016年6月推出外溢保單，鼓勵被保險人落實或提升自身健康管理觀念及行為。（圖／達志示意圖）
記者紀佳妘／台北報導
運動省保費近年來被受國人青睞，根據金管會保險局今（5）日統計，2019年外溢保單銷售件數達6萬117件、初年度保費收入達8億1,646萬元，較上年同期增加4.64倍、8.23倍，其中健走型保單賣最好，保費收入就占8成。
金管會目前已核准及備查8家壽險公司，共28張具外溢效果的保險商品，2019年新契約銷售件數為6萬117件，較2018年同期的1萬645件，增加464%；初年度保費收入約8億1,646萬元，較上年同期的8,839萬元，增加823%。
觀察具外溢效果保險商品的表現，其中以「健走型外溢保單」賣最好，2019全年新契約件數為2萬9,172件，年增2萬6,935件，而初年度保費收入達6億5,554萬元，年增6億610萬元，保費收入就占8成。
從健走型外溢保單的初年度保費來看，前3名的保險公司依序為國泰人壽（約5.7億元）、南山人壽（約1.4億元）以及新光人壽（約8,800萬元）。
另外，金管會目前已核准及備查6家壽險公司，共16張實物給付型保險商品，2019年新契約銷售件數為18萬1,775件，較2018年同期的15萬1,860件增加；初年度保費收入約2,779萬元，則較去年同期的3,129萬元減少。
實物給付型外溢保單2019年保費收入前3大保險公司，第一名是國泰人壽，緊追在後的是富邦人壽，排名第三的是台灣人壽；觀察初年度保費收入性質，醫療服務類型金額為2,855萬元，殯葬服務類型也有989萬元，至於健康管理服務類型僅2萬元。
金管會表示，將持續鼓勵保險業者研發具外溢效果的健康管理保險商品，以及實物給付型保險商品，以提升國人對於健康管理的重視，並滿足保戶保險的多樣需求。
----------------------------------------------------------------------

[32] 北市商業會：政府應主動減免營所稅　非常時期助企業度難關　　

▲北市商業會理事長吳發添，呼籲政府採取稅賦減抵損失措施，幫助企業共度難關。（資料照／台北市商業會提供）
記者李錦奇／台北報導
武漢疫情沒有趨緩跡象，對於企業而言，業績面臨下滑衝擊之際，還得照顧員工生計，蠟燭兩頭燒。對此，北市商業會理事長吳發添呼籲，在此非常

▲武漢肺炎疫情持續延燒，不僅口罩一罩難求，防疫產品如消毒酒精也搶翻天，造成市面上大缺貨。（圖／記者周宸亘攝）
記者林淑慧／台北報導
為因應武漢肺炎防疫需求，政府統計國內消毒酒精仍有40萬瓶需求缺口，台糖今（5）日表示，目前已開三班制趕工中，產能已較往前提升五成，也將召集民間製藥業者洽談代工，預計新增百萬瓶產能，以滿足國內防疫消毒需求。
武漢肺炎疫情持續延燒，不僅口罩一罩難求，防疫產品如消毒酒精搶翻天，大批民眾至藥局、美妝店排隊購買酒精。台糖表示，民眾除了戴口罩避免感染外，勤洗手及消毒亦是阻擋新型冠狀病毒傳播的重要守則，而濃度75%的酒精更可達到徹底殺菌的功能。
針對目前酒精產能，台糖表示，目前產線均已三班制趕工中，由於台糖生產的酒精交由經銷商銷售，經銷商每天都會到台糖工廠運貨，目前台糖旗下20間蜜鄰超市也販售500毫升酒精，每人限購一瓶，每瓶價格維持新台幣62元。
▲台糖指出，酒精濃度介於70%至78%都可達到殺菌的效果，民眾可以3:1的比例將95%精製酒精稀釋為消毒用酒精，全民一起抗毒做防疫。（圖／台糖提供）
台糖指出，酒精濃度介於70%至78%都可達到殺菌的效果，民眾可以3:1的比例將95%精製酒精稀釋為消毒用酒精，意即以3份台糖精製酒精與1份蒸餾水或煮沸過冷水混合，即可調配出濃度70%至78%的酒精，達到消毒殺菌的效果，操作簡單、價格便宜且有效。
台糖提醒，不少民眾出入公共場所時會隨身攜帶噴霧式酒精消毒，但因酒精具有一定程度的可燃性，因此民眾在防疫之餘，也要格外注意消防安全。
----------------------------------------------------------------------

[44] 鴻海應邀召開線上法說　說明武漢肺炎疫情因應狀況

▲鴻海土城總部。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
中國大陸武漢疫情延燒，由於鴻海（2317）iPhone組裝生產大本營鄭州市也宣布封閉式管理，市場憂心影響量產出貨，對此，鴻海今（5）日重訊公告，預計在明（6）日應美林證券之邀參加電話會議說明公司營運概況，激勵早盤股價由黑翻紅，力守平盤之間微幅震盪。
鴻海今日受到武漢肺炎疫情衝擊，盤中股價一度跌0.61%，但在鴻海重訊公告即將在明日應美林證券之邀參加電話會議說明公司營運概況之後，股價由黑翻紅，最高來到82.7元，漲逾0.6%，緊接

記者周康玉／台北報導
南亞科（2408）公布1月份自結合併營收為45億元，較上月增加3.92%，較去年同期增加5.68%，已經是自去年11月以來，連三月增長。展望第一季，南亞科維持1月時法說會看法，今年DRAM位元產出增加有限，供需可望更健康，第一季已經看到好轉，第2季又將優於第1季。
有關新型冠狀病毒疫情影響，南亞科表示，公司已啟動各項內部防疫措施，並優先配合政府單位的防疫規定。此外，南亞科在中國大陸並無生產據點，生產製造不受影響；目前已與原物料供應商及外包商確認供應及營運正常。從全球銷售觀察，目前不受影響，公司將持續密切觀察事件後續發展，並採取必要措施以確保整體營運及出貨正常。
針對武漢肺炎防疫措施，南亞科表示，公司已成立防疫應變小組並已採取嚴密防疫措施，提醒同仁需自主健康管理及照護，避免出差至疫區，並以視訊或電話會議取代拜訪。員工、廠商及訪客進入本公司廠區與辦公室前一律由專人量測體溫；截至目前為止，全體員工、合作廠商、來賓訪客等皆無異常。
此外，對於勞動部宣布「防疫照顧假」一事，南亞科表示，公司配合政府政策依規定辦理；然而是否給薪，目前內部仍在研議中。
----------------------------------------------------------------------

[58] 美股連續2天開盤大漲！30分鐘狂飆450點　特斯拉股價破900美元

記者王致凱／綜合報導
市場對新型冠狀病毒疫情憂慮放緩。美股連續第二天上漲，道瓊工業指數開盤後30分鐘飆漲450點或1.6％，報28,840.20點；標普500指數和那斯達克綜合指數也上漲1.4％。
道瓊指數周五（1月31日）收盤暴跌603點或2.1％，但周一（1日）反彈上漲144點或0.5％。周二美股漲勢持續，包括蘋果上漲1.9％，英偉達（Nvidia）和美光（Micron）漲幅均超過2％。
此外，特斯拉以超過17％的漲幅帶動那斯達克上漲。特斯拉股價今日飆漲至900美元以上，為六年來表現最亮眼一次。投資人巴倫（Ron Baron）表示，特斯拉有望實現1兆美元營收。
----------------------------------------------------------------------

[59] 防堵武漢肺炎疫情　一銀、彰銀給員工14天有薪「防疫照顧假」

▲因應延後開學

▲興富發大方祭出40週年感恩回饋優惠方案，上網預約早鳥9折買房大優惠。（圖／興富發建設提供，下同）
圖、文／興富發建設
不用中樂透  「早鳥預約」開春大紅包
早鳥可以預約買什麼，預約買高鐵票、買演唱會的票，預約尾牙餐廳、預約年節禮盒、預約國外高檔旅行團，但這些回饋都抵擋不了，金鼠年最強的回饋：「上網預約買房，享9折優惠。」網友驚呼：「省下了錢，讓你鼠錢鼠不完啊！」也有網友說：「已預約啦，還沒預約的是在哈囉？」
破天荒！上網預約早鳥9折限量回饋中 
全台建商龍頭興富發集團欣逢40週年，日前慶祝獲得第4屆爆米花棒球聯盟冠軍，興富發總裁鄭欽天在集團尾牙時，大方祭出40週年感恩回饋優惠方案，上網預約早鳥9折買房大優惠，讓全台灣購屋族湧入網站，預約盛況空前。
凡即日起至2020/02/29（六），前往興富發官方賞屋平台「早鳥專區」完成預約登記，享興富發子公司巨豐管理代銷之北中南五大全新個案9折優惠，包括台中七期案、桃園青溪案、台南漁光案、高雄農十六案、高雄民生臻愛2，簽約完成加碼再贈時下最夯「電動自行車一台」。
大手筆回饋就是了解新一代購屋族的辛苦，也讓消費者荷包省一大筆，有網友就試算若房價為1000萬，早鳥預約享9折優惠，等於省下100萬元的購屋金，「100萬大紅包是樂透彩過年才會加開的獎，抽中才會有，這個線上預約就會有了啦！」
買房當存錢，有家才有安全感！
而對於買房與否，許多購屋族也還在觀望，租房派認為房價太高，租比買便宜很多，還不用背沉重房貸，維持生活品質；買房派則認為與其把房租繳給房東，還不如把錢拿來買房，幾年後就擁有一間屬於自己的房子。
興富發集團鄭欽天就曾公開表示，年輕人一定要買房，租房子只能當一輩子屋奴，「有家才有安全感！」，寧可2～10年不要買車，且車子只會折舊，房子還會增值，買車馬上賠錢、買房是開始存錢。也不用擔心付不起，過去是存好錢買房，現在低利率環境，加上低自備、輕鬆付款，可以先買房當存錢。
而興富發集團也提供優於其他建商的低自備，讓想買房的人有了機會，利用過去固定儲蓄和投資理財資金，再加上剛到手的年終獎金，一鼓作氣將這筆錢換成不動產，錢沒被胡亂花掉，最後再加上最新提供的北中南五大全新個案9折優惠，精打細算省下不少，房子買了，心裡才會變得踏實！
------------------------------------------------------

▲ 台股全面逐筆交易與盤中零股交易制度將於今年實施。（圖／記者湯興漢攝）
記者李瑞瑾／台北報導
台股逐筆交易將於3月23日上線，再6個月後也將實施盤中零股交易。券商公會理事長賀鳴珩認為，這兩項是台灣證券市場必要的基礎工程，零股交易更是金融創新關鍵。
賀鳴珩指出，目前電子下單及全面化作業已經普及，實施盤中零股交易的時機已經成熟，將在逐筆上線後的6個月實施，但需要搭配幾個前提，提升交易便利，零股交易才會成功。證交所也已在去年底公告新增額度100萬以内的新客戶，可手持身分證件拍照搭配證券商分戶帳指定出金帳戶辦理網路開戶。
▲ 券商公會理事長賀鳴珩。（圖／記者湯興漢攝）
此外，亦推廣開立分戶帳，讓投資人可以在證券商同時完成開立證券戶與分戶帳帳戶，不用再到銀行開交割專戶。並持續鼓勵從開戶、下單、帳務到集保都要全面e化，才能降低成本。
賀鳴珩認為，撮合間隔時間縮短才能滿足投資人交易習慣、滅少交易糾紛，他表示，零股交易希望能將3分鐘撮合一次改為5秒鐘撮合一次，但也要待逐筆交易上線且穩定了之後，在來研議是否有可能將零股交易撮合時間縮短。
券商公會表示，為利發展金融科技及普惠金融，將持續研議優化盤中零股交易相關配套措施，如：交易撮合頻率、引進造市機制、強化投資人下單系統辨識、研議電子憑證收費方案等，以積極吸引年輕族群及小資族參與，對開發新客群、證券業AI、產品創新等目標發揮關鍵影響，亦可充分與國際制度平行接軌。
另為讓證券從業人員及投資人瞭解盤中零股交易與現行盤後零股交易不同，券商公會將與證交所、櫃買中心共同加強從業人員的教育訓練，以利第一線的從業人員與投資人順利執行盤中零股交易。
----------------------------------------------------------------------

[83] 兩廳院售票系統升級！選位可「視野預覽」、AI智慧學習觀眾喜好

記者林育綾／台北報導
耗時一年多的研發，「兩廳院售票系統」將全面「打掉重練」，升級為「OpenTix」文化生活電商平台（暫名），不僅選位更便捷，還有「視野預覽」功能，加上更具智慧的全文檢索，方便觀眾找尋節目，還有AI 智慧學習模式，能瞭解觀眾對表演和選位的喜好，未來自動推薦。全新的系統預計4月至5月左右開放公測，期望能在今年第四季上線！
▲升級的票務系統，未來有全文檢索功能，找尋喜愛的節目更

▲開發金今日公告，開發金總經理、中國人壽董事長王銘陽將在3月4日退休。（圖／記者湯興漢攝）
記者吳靜君／台北報導
開發金今（4）日公告，開發金總經理、中壽董事長董事長王銘陽退休，其法人董事職缺待基捷投資股份有限公司另行指派，並經由董事會選出新任總經理接替。根據了解，王銘陽雖然外表十分年輕，今年仍未滿64歲，而且目前開發金日前才向金管會申請延長併中壽的時程、以及及中壽、開發金的新總部大樓即將落成，選擇此時退休引發外界好奇。
曾經在南山人壽服務，從基層的投資部做起，歷經主任、經理、副總經理等職務，在壽險業相當受矚目的菁英，也在中壽任總經理、在開發金購買中壽35%股權，轉任開發金總經理、中壽董事長。其學歷為美國德州大學達拉斯分校管理科學研究所碩士。
根據開發金的公告，目前尚未決定接任人選，開發金表示，總經理的職缺要等待基捷投資另行指派；中壽的公告則是，要等待開發金決定。
----------------------------------------------------------------------

[94] 外傳口罩原料短缺？經部：原料供應無虞、新產線三月起陸續上線

▲武漢肺炎疫情持續延燒，造成全球口罩搶購熱潮。（圖／路透）
記者林淑慧／台北報導
武漢肺炎疫情持續升溫，全球瘋搶口罩，外傳台灣口罩的不織布原料恐將短缺；對此，經濟部長沈榮津召集口罩業者與會，業者大多表示原料供應無虞，將力拚新產線於三月起陸續上線，以滿足國內口罩需求的缺口。
經濟部表示，政府對口罩生產的原料一直都有掌握，從PP聚丙烯（不織布原料）、到口罩內層ES、中間層熔噴布、外層紡黏，國內都有生產，經濟部將隨時注意原物料的供應情況。
「（經濟）部長昨天的指令很明確，就是衝高產能、確保物料。所以新產線開始後，部長請林次（經濟部常務次長林全能）盤點後續的原物料供應無虞，他也會再找業者討論。」經濟部官員指出。
依據經濟部最新統計，目前國內口罩日產能為400萬片，官員指出，目前已經有不少製造商，採取全天候輪班生產的方式製造口罩，而本月政府斥資2億佈建的 62條口罩產線也將陸續上線，國內產能可望逐步提高。
----------------------------------------------------------------------

[95] 和泰產險董事長由「前全球人壽總經理」張永固出任